# **Reading User's Data**

In [5]:
import pandas as pd
import numpy as np

In [6]:
def read_data(uuids):


   user_data_file='%s.features_labels.csv'%uuids;
   df = pd.read_csv(user_data_file);

   # The first column should be timestamp:
   assert df.columns[0] == 'timestamp';
   #  The last column should be label_source:
   assert df.columns[-1] == 'label_source';
   for (ci,col) in enumerate(df.columns):
        if col.startswith('label:'):
            first_label_ind = ci;
            break;
        pass;
   df.columns = df.columns.str.replace('label:', '')

   # Feature columns come after timestamp and before the labels:
   feature_names = df.columns[1:first_label_ind];
   # Then come the labels, till the one-before-last column:
   label_names = df.columns[first_label_ind:-1];

   n_features = len(feature_names);
   timestampe= df.iloc[:,0]

   X = df.iloc[:,1:(n_features+1)]
   X = X.fillna(0)
   trinary_labels_mat = df.iloc[:,(n_features+1):-1]; # This should have values of either 0., 1. or NaN
   trinary_labels_mat = trinary_labels_mat.fillna(0)
   
   #Deleting the columns:

   #for col in trinary_labels_mat.columns:
   #    if trinary_labels_mat[col].sum(axis = 0)<=0 :
   #       del trinary_labels_mat[col]

   M = pd.isna(trinary_labels_mat)
   Y = np.where(M,0,trinary_labels_mat) > 0.;

   return X , Y , M , trinary_labels_mat, feature_names, label_names, timestampe,n_features ;


In [7]:
uuids='0E6184E1-90C0-48EE-B25A-F1ECB7B9714E';
(X , Y , M , trinary_labels_mat, feature_names, label_names, timestampe, n_features) = read_data(uuids);


In [ ]:
trinary_labels_mat

,LYING_DOWN,SITTING,FIX_walking,FIX_running,BICYCLING,SLEEPING,LAB_WORK,IN_CLASS,IN_A_MEETING,LOC_main_workplace,OR_indoors,OR_outside,IN_A_CAR,ON_A_BUS,DRIVE_-_I_M_THE_DRIVER,DRIVE_-_I_M_A_PASSENGER,LOC_home,FIX_restaurant,PHONE_IN_POCKET,OR_exercise,COOKING,SHOPPING,STROLLING,DRINKING__ALCOHOL_,BATHING_-_SHOWER,CLEANING,DOING_LAUNDRY,WASHING_DISHES,WATCHING_TV,SURFING_THE_INTERNET,AT_A_PARTY,AT_A_BAR,LOC_beach,SINGING,TALKING,COMPUTER_WORK,EATING,TOILET,GROOMING,DRESSING,AT_THE_GYM,STAIRS_-_GOING_UP,STAIRS_-_GOING_DOWN,ELEVATOR,OR_standing,AT_SCHOOL,PHONE_IN_HAND,PHONE_IN_BAG,PHONE_ON_TABLE,WITH_CO-WORKERS,WITH_FRIENDS
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Deleting the missing examples**

In [ ]:
trinary_labels_mat['sum'] =np.sum(trinary_labels_mat, axis=1)
# (  if (trinary_labels_mat['sum'] == null): trinary_labels_mat)
trinary_labels_mat=trinary_labels_mat[trinary_labels_mat['sum']!=0]
trinary_labels_mat=trinary_labels_mat.drop(['sum'],axis=1)
df = pd.concat([trinary_labels_mat,X ], axis=1, join='inner')
X=df.iloc[:,51:]

In [ ]:
row_sum = np.sum(trinary_labels_mat, axis=1)
print(row_sum)
#yyy= row_sum.value_counts()[0]
#print(yyy)

In [ ]:
print(X.shape);
print(Y.shape);
print(M.shape);
print(trinary_labels_mat);
print(timestampe.shape);
print(label_names.shape);
print(feature_names.shape);

**Sum of labels**

In [ ]:
n_examples_per_label = np.sum(trinary_labels_mat,axis=0);
labels_and_counts = zip(label_names,n_examples_per_label);
sorted_labels_and_counts = sorted(labels_and_counts,reverse=True,key=lambda pair:pair[1]);
print ("How many examples does this user have for each contex-label:");
print ("-"*50);
for (label,count) in sorted_labels_and_counts:
    print ("label %s - %d minutes" % (label,count));
    pass;

**Get sensor names from features**

In [ ]:
def get_sensor_names_from_features(feature_names):
    feat_sensor_names = np.array([None for feat in feature_names]);
    for (fi,feat) in enumerate(feature_names):
        if feat.startswith('raw_acc'):
            feat_sensor_names[fi] = 'Acc';
            pass;
        elif feat.startswith('proc_gyro'):
            feat_sensor_names[fi] = 'Gyro';
            pass;
        elif feat.startswith('raw_magnet'):
            feat_sensor_names[fi] = 'Magnet';
            pass;
        elif feat.startswith('watch_acceleration'):
            feat_sensor_names[fi] = 'WAcc';
            pass;
        elif feat.startswith('watch_heading'):
            feat_sensor_names[fi] = 'Compass';
            pass;
        elif feat.startswith('location'):
            feat_sensor_names[fi] = 'Loc';
            pass;
        elif feat.startswith('location_quick_features'):
            feat_sensor_names[fi] = 'Loc';
            pass;
        elif feat.startswith('audio_naive'):
            feat_sensor_names[fi] = 'Aud';
            pass;
        elif feat.startswith('audio_properties'):
            feat_sensor_names[fi] = 'AP';
            pass;
        elif feat.startswith('discrete'):
            feat_sensor_names[fi] = 'PS';
            pass;
        elif feat.startswith('lf_measurements'):
            feat_sensor_names[fi] = 'LF';
            pass;
        else:
            raise ValueError("!!! Unsupported feature name: %s" % feat);

        pass;

    return feat_sensor_names;

In [ ]:
feat_sensor_names = get_sensor_names_from_features(feature_names);

for (fi,feature) in enumerate(feature_names):
    print("%3d) %s %s" % (fi,feat_sensor_names[fi].ljust(10),feature));
    pass;


# **Oversampling technique of minority labels (MLSMOTE)**

In [ ]:
import random
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
def get_tail_label(df):
    """
    Give tail label colums of the given target dataframe
    
    args
    df: pandas.DataFrame, target label df whose tail label has to identified
    
    return
    tail_label: list, a list containing column name of all the tail label
    """
    columns = df.columns
    n = len(columns)
    irpl = np.zeros(n)
    for column in range(n):
         irpl[column] = df[columns[column]].value_counts()[1]
    irpl = max(irpl)/irpl
    print(irpl)
    mir = np.mean(irpl, axis=0)
    print(mir)
    tail_label = []
    for i in range(n):
        if irpl[i] > mir:
          tail_label.append(columns[i])
    return tail_label

def get_index(df):
  """
  give the index of all tail_label rows
  args
  df: pandas.DataFrame, target label df from which index for tail label has to identified
    
  return
  index: list, a list containing index number of all the tail label
  """
  tail_labels = get_tail_label(df)
  print(tail_labels)

  index = set()
  for tail_label in tail_labels:
    sub_index = set(df[df[tail_label]==1].index)
    index = index.union(sub_index)
  return list(index)

def get_minority_instace(X, y):
    """
    Give minority dataframe containing all the tail labels
    
    args
    X: pandas.DataFrame, the feature vector dataframe
    y: pandas.DataFrame, the target vector dataframe
    
    return
    X_sub: pandas.DataFrame, the feature vector minority dataframe
    y_sub: pandas.DataFrame, the target vector minority dataframe
    """
    index = get_index(y)
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

def nearest_neighbour(X):
    """
    Give index of 5 nearest neighbor of all the instance
    
    args
    X: np.array, array whose nearest neighbor has to find
    
    return
    indices: list of list, index of 5 NN of each element in X
    """
    nbs=NearestNeighbors(n_neighbors=5,metric='euclidean',algorithm='kd_tree').fit(X)
    euclidean,indices= nbs.kneighbors(X)
    return indices

def MLSMOTE(X,y, n_sample):
    """
    Give the augmented data using MLSMOTE algorithm
    
    args
    X: pandas.DataFrame, input vector DataFrame
    y: pandas.DataFrame, feature vector dataframe
    n_sample: int, number of newly generated sample
    
    return
    new_X: pandas.DataFrame, augmented feature vector data
    target: pandas.DataFrame, augmented target vector data
    """
    indices2 = nearest_neighbour(X)
    n = len(indices2)
    new_X = np.zeros((n_sample, X.shape[1]))
    target = np.zeros((n_sample, y.shape[1]))
    for i in range(n_sample):
        reference = random.randint(0,n-1)
        neighbour = random.choice(indices2[reference,1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val>2 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbour,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    new_X = pd.concat([X, new_X], axis=0)
    target = pd.concat([y, target], axis=0)
    return new_X, target

In [ ]:
"""
    main function to use the MLSMOTE
"""
X_sub, y_sub = get_minority_instace(X, trinary_labels_mat )   #Getting minority instance of that datframe
X_res,y_res =MLSMOTE(X_sub, y_sub, 101176 )     #Applying MLSMOTE to augment the dataframe

**Adding LeakyRelu activation function for MLPClassifier**

In [ ]:
from sklearn.neural_network.multilayer_perceptron import(ACTIVATIONS, DERIVATIVES, MLPClassifier)
#import numpy as np
def leakyrelu(X, alpha =0.1):
    np.clip(X, alpha*X, np.finfo(X.dtype).max, out=X)   
    return X

def inplace_leakyrelu_derivative(Z, delta, alpha=0.01):
    delta[Z < 0] = alpha


ACTIVATIONS['leakyrelu']= leakyrelu
DERIVATIVES['leakyrelu']= inplace_leakyrelu_derivative

**Standardization of Features**

In [ ]:
def project_features_to_selected_sensors(X,feat_sensor_names,sensors_to_use):
    use_feature = np.zeros(len(feat_sensor_names),dtype=bool);
    for sensor in sensors_to_use:
        is_from_sensor = (feat_sensor_names == sensor);
        use_feature = np.logical_or(use_feature,is_from_sensor);

        pass;
    
    X = X.iloc[:,use_feature];
    return X;

def estimate_standardization_params(X_train):
    mean_vec = np.nanmean(X_train,axis=0);
    
    std_vec = np.nanstd(X_train,axis=0);
    return (mean_vec,std_vec);

def standardize_features(X,mean_vec,std_vec):
    # Subtract the mean, to centralize all features around zero:
    X_centralized = X - mean_vec.reshape((1,-1));
    # Divide by the standard deviation, to get unit-variance for all features:
    # * Avoid dividing by zero, in case some feature had estimate of zero variance
    normalizers = np.where(std_vec > 0., std_vec, 1.).reshape((1,-1));
    X_standard = X_centralized / normalizers;
    return X_standard;



**Normalization**

In [ ]:
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
def min_max_scaler(A):
  scaler = minmax_scale(A,feature_range=(-1,1), axis=1)
  #scaler = MinMaxScaler()
  #scaler = scaler.fit_transform(A)
  #scaler =scaler.transform(A)
  return scaler;

# **Model Training**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score ,GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier

def train_model(X_train,Y_train,feat_sensor_names,label_names,sensors_to_use):
     
    X_train = project_features_to_selected_sensors(X_train,feat_sensor_names,sensors_to_use);
    
    #X_train = min_max_scaler(X_train)
    
    (mean_vec,std_vec) = estimate_standardization_params(X_train);
    X_train = standardize_features(X_train,mean_vec,std_vec);
    

    X_train[np.isnan(X_train)] = 0.;

    #print("== Training with %d examples. For label we have %d positive and %d negative examples." % \
     #     (len(trinary_labels_mat),sum(y),sum(np.logical_not(y))) );
   
   
   #"'Logistic regression model:'
   #'Parameter Tuning for Logistic regression:'
   #parametars= {'C':[0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100]},

   #clf = LogisticRegression(class_weight='balanced',max_iter=200)
    

   #'Spliting the Data into Train and Test set:'
   #X_train1, X_test1, Y_train1, Y_test1 = train_test_split(x_train, Y_train, test_size= 0.3)
    
   #'Parameter Tuning for MLPClassifier:'

   #parameters = {'hidden_layer_sizes':[(2,),(4,),(8,),(16,),(32,)], 
                  #'alpha':[0.0001,0.0005,0.001,0.005,0.01,0.05,0.1]}
   #'Gridsearch:'
   #grid_search = GridSearchCV(clf, parameters, cv=5)
   #grid_search.fit(X_train1,Y_train1);
   #print(grid_search.best_params_)
    

   #'principal component analysis (PCA):'
   # from sklearn.decomposition import PCA
   # pca = PCA(n_components=128)
   # pca.fit(X_train)
   # X_pca = pca.transform(X_train)
   # print(X_train.shape)
   # print(X_pca.shape)

   #'Muli-task MLP:'
    clf = MLPClassifier (hidden_layer_sizes = (2,), activation='leakyrelu',random_state=0, max_iter = 40, solver = 'sgd', 
                       momentum=0.5, learning_rate='invscaling', alpha=0.01,power_t=0.5, learning_rate_init=0.1, batch_size=300)
   #clf_model=clf.fit(X_pca, Y_train)
    clf_model=clf.fit(X_train, Y_train)
    
   #'MLP per label:'
   #clf_out = MultiOutputClassifier(clf)
   #clf_model = clf_out.fit(X_train,Y_train) 
   

    model = {\
            'sensors_to_use':sensors_to_use,\
            'mean_vec':mean_vec,\
            'std_vec':std_vec,\
            'clf_model':clf_model};
    return model;

In [ ]:
print(X.shape)
print(trinary_labels_mat)

In [ ]:
sensors_to_use = ['Acc','WAcc','Gyro','Loc','Aud','PS'];

model = train_model(X,trinary_labels_mat,feat_sensor_names,label_names,sensors_to_use);
#model = train_model(X_res,y_res,feat_sensor_names,label_names,sensors_to_use);

In [ ]:
def test_model(X_test,Y_test,feat_sensor_names,label_names,model):
    # Project the feature matrix to the features from the sensors that the classifier is based on:
    X_test = project_features_to_selected_sensors(X_test,feat_sensor_names,model['sensors_to_use']);
    print("== Projected the features to %d features from the sensors: %s" % (X_test.shape[1],', '.join(model['sensors_to_use'])));
    
    # We should standardize the features the same way the train data was standardized:
    
    (mean_vec,std_vec) = estimate_standardization_params(X_test);
    X_test = standardize_features(X_test,mean_vec,std_vec);
    
    #X_test = min_max_scaler(X_test)
    
    y = Y_test;
    print(y.shape)

    # Do the same treatment for missing features as done to the training data:
    X_test[np.isnan(X_test)] = 0.;
    
    #print("== Testing with %d examples. For label '%s' we have %d positive and %d negative examples." % \
         # (len(y),get_label_pretty_name(label_names),sum(y),sum(np.logical_not(y))) );
    
    # Preform the prediction:
    y_pred = model['clf_model'].predict(X_test);
    
    #from sklearn.metrics import multilabel_confusion_matrix
    #confusion_matrix= multilabel_confusion_matrix(y, y_pred)
    #print (confusion_matrix)
    # Naive accuracy (correct classification rate):
    accuracy = np.mean(y_pred == y);
    accuracy=np.mean(accuracy)

    y = np.array(y);
    y_pred= np.array(y_pred);

    # Count occorrences of true-positive, true-negative, false-positive, and false-negative:
    tp = np.sum(np.logical_and(y_pred,y));
    tn = np.sum(np.logical_and(np.logical_not(y_pred),np.logical_not(y)));
    fp = np.sum(np.logical_and(y_pred,np.logical_not(y)));
    fn = np.sum(np.logical_and(np.logical_not(y_pred),y));
    
    # Sensitivity (=recall=true positive rate) and Specificity (=true negative rate):
    sensitivity = float(tp) / (tp+fn);
    specificity = float(tn) / (tn+fp);
    
    # Balanced accuracy is a more fair replacement for the naive accuracy:
    balanced_accuracy = (sensitivity + specificity) / 2.;
    
    # Precision:
    # Beware from this metric, since it may be too sensitive to rare labels.
    # In the ExtraSensory Dataset, there is large skew among the positive and negative classes,
    # and for each label the pos/neg ratio is different.
    # This can cause undesirable and misleading results when averaging precision across different labels.
    precision = float(tp) / (tp+fp);

    print("-"*10);
    print('Accuracy*:         %.2f' % accuracy);
    print('Sensitivity (TPR): %.2f' % sensitivity);
    print('Specificity (TNR): %.2f' % specificity);
    print('Balanced accuracy: %.2f' % balanced_accuracy);
    print('Precision**:       %.2f' % precision);
    print("-"*10);


    return;

In [ ]:
test_model(X,trinary_labels_mat,feat_sensor_names,label_names,model);

== Projected the features to 175 features from the sensors: Acc, WAcc, Gyro, Loc, Aud, PS
(101176, 51)
----------
Accuracy*:         0.95
Sensitivity (TPR): 0.45
Specificity (TNR): 0.99
Balanced accuracy: 0.72
Precision**:       0.77
----------


**Reading new User's Data**

In [ ]:
uuiids='fold_4_test';
(X1 , Y1 , M1, trinary_labels_mat_test, feature_names1, label_names1, timestampe1, n_features) = read_data(uuiids);


In [ ]:
X1.shape , trinary_labels_mat_test.shape 

((29137, 225), (29137, 51))

In [ ]:
trinary_labels_mat_test['sum'] =np.sum(trinary_labels_mat_test, axis=1)
# (  if (trinary_labels_mat['sum'] == null): trinary_labels_mat)
trinary_labels_mat_test=trinary_labels_mat_test[trinary_labels_mat_test['sum']!=0]
trinary_labels_mat_test=trinary_labels_mat_test.drop(['sum'],axis=1)
df = pd.concat([trinary_labels_mat_test,X1 ], axis=1, join='inner')
X1=df.iloc[:,51:]

**Testing on new User's data**

In [ ]:
test_model(X1,trinary_labels_mat_test,feat_sensor_names,label_names,model);
print(X1.shape, Y1.shape)

== Projected the features to 175 features from the sensors: Acc, WAcc, Gyro, Loc, Aud, PS
(29137, 51)
----------
Accuracy*:         0.95
Sensitivity (TPR): 0.47
Specificity (TNR): 0.99
Balanced accuracy: 0.73
Precision**:       0.74
----------
(29137, 225) (29137, 51)
